# Data source

Data for download: https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz

Resources used as reference:
1) Training the image classifier to recognize different species of flowers:
https://www.kaggle.com/dtosidis/flower-classifier-tensorflow
  
2) Loading and preprocessing an image dataset
https://www.tensorflow.org/tutorials/load_data/images

3) Data augmentation
https://www.tensorflow.org/tutorials/images/data_augmentation

4) Image classification
https://www.tensorflow.org/tutorials/images/classification


In [24]:
# Dependencies
import matplotlib.pyplot as plt
%matplotlib inline

import os
import numpy as np
import tensorflow as tf

import PIL
import PIL.Image

import pathlib

os.environ['KMP_DUPLICATE_LIB_OK']='True'

from tensorflow import keras
from tensorflow.keras.preprocessing import image
import tensorflow_hub as hub

from tensorflow.keras.applications.vgg19 import (
    VGG19, 
    preprocess_input, 
    decode_predictions
)

In [25]:
!pwd

/c/ShankersDocs/EDUCATION/RICE_Bootcamp_DataAnalytics/FinalProject_Img_Recognition_Flowers/Final_RICEproject_ImageRecognition_flowers


### Total images in the dataset

In [26]:
data_dir = 'flower_photos'
data_dir = pathlib.Path(data_dir)
image_count = len(list(data_dir.glob('*/*.jpg')))
# print(image_count)

## Image Classification
https://www.tensorflow.org/tutorials/images/classification?hl=zh-tw

In [27]:

batch_size = 32
img_height = 224
img_width = 224

## Generating datasets
https://keras.io/examples/vision/image_classification_from_scratch/

### Generating a training dataset

In [28]:
# When the subset below is defined as "training" the 0.2 validation split takes 80% of the data as the training set

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'flower_photos',
  validation_split=0.1,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3670 files belonging to 5 classes.
Using 3303 files for training.


### Generating a validation dataset

In [29]:
# When the subset below is defined as "validation" the 0.1 validation split takes 10% of the data as the validation set

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'flower_photos',
  validation_split=0.1,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3670 files belonging to 5 classes.
Using 367 files for validation.


### Generating a test dataset

In [30]:
# When the subset below is defined as "validation" the 0.1 validation split takes 10% of the data as the test set

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'flower_photos',
  validation_split=0.1,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3670 files belonging to 5 classes.
Using 367 files for validation.


### Class names

In [31]:
class_names = train_ds.class_names
print(class_names)

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


### Rescaling the data
https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/Rescaling


In [32]:
from tensorflow.keras import layers

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

### Normalizing the data (trainign and validation datasets)

In [33]:
normalized_train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
normalized_val_ds =  val_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_train_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image)) 

0.0 1.0


### Autotune is done to cache data and make processing and resource mgmt more effieicient

In [34]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [35]:
normalized_train_ds = normalized_train_ds.cache().prefetch(buffer_size=AUTOTUNE)
normalized_val_ds = normalized_val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [36]:
# num_classes = 5
num_classes = len(class_names)
num_classes

5

## Model 1 (Sequential Model)
https://www.tensorflow.org/guide/keras/sequential_model

In [21]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(img_height, img_width, 3)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(num_classes, activation='softmax')
])


In [22]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

In [23]:
model.fit(
    train_ds,
    validation_data = val_ds, 
    epochs=16
)

Epoch 1/16
92/92 [==============================] - 13s 137ms/step - loss: 5208.3359 - accuracy: 0.3219 - val_loss: 1223.1353 - val_accuracy: 0.4223
Epoch 2/16
92/92 [==============================] - 6s 67ms/step - loss: 1124.1339 - accuracy: 0.3910 - val_loss: 721.5298 - val_accuracy: 0.4360
Epoch 3/16
92/92 [==============================] - 6s 69ms/step - loss: 989.1677 - accuracy: 0.4142 - val_loss: 454.4877 - val_accuracy: 0.4087
Epoch 4/16
92/92 [==============================] - 6s 66ms/step - loss: 734.0304 - accuracy: 0.4240 - val_loss: 737.4456 - val_accuracy: 0.3924
Epoch 5/16
92/92 [==============================] - 6s 67ms/step - loss: 693.9451 - accuracy: 0.4315 - val_loss: 767.1343 - val_accuracy: 0.3569
Epoch 6/16
92/92 [==============================] - 6s 69ms/step - loss: 516.3157 - accuracy: 0.4244 - val_loss: 578.7139 - val_accuracy: 0.3733
Epoch 7/16
92/92 [==============================] - 7s 71ms/step - loss: 421.5355 - accuracy: 0.4264 - val_loss: 374.1722 - v

## Model 1 (Sequential Model) Epoch sensitivity

In [24]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(img_height, img_width, 3)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(num_classes, activation='softmax')
])


In [25]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

In [30]:
model.fit(
    train_ds,
    validation_data = val_ds, 
    epochs=6
)

Epoch 1/6
92/92 [==============================] - 6s 68ms/step - loss: 3109.6846 - accuracy: 0.3198 - val_loss: 2002.3905 - val_accuracy: 0.3324
Epoch 2/6
92/92 [==============================] - 6s 64ms/step - loss: 1059.0967 - accuracy: 0.3740 - val_loss: 1328.4467 - val_accuracy: 0.2425
Epoch 3/6
92/92 [==============================] - 6s 65ms/step - loss: 676.3892 - accuracy: 0.4217 - val_loss: 1104.6982 - val_accuracy: 0.3324
Epoch 4/6
92/92 [==============================] - 6s 67ms/step - loss: 480.6086 - accuracy: 0.4561 - val_loss: 832.1366 - val_accuracy: 0.3569
Epoch 5/6
92/92 [==============================] - 6s 66ms/step - loss: 406.3172 - accuracy: 0.4935 - val_loss: 830.6475 - val_accuracy: 0.3706
Epoch 6/6
92/92 [==============================] - 6s 66ms/step - loss: 464.5512 - accuracy: 0.4816 - val_loss: 1717.7488 - val_accuracy: 0.2834


## Model 1a (Sequential Model) With Normalized data

In [40]:
model_1a = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(img_height, img_width, 3)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(num_classes, activation='softmax')
])


In [41]:
model_1a.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

In [42]:
model_1a.fit(
    normalized_train_ds,
    validation_data = normalized_val_ds, 
    epochs=6
)

Epoch 1/6
92/92 [==============================] - 6s 67ms/step - loss: 9.3329 - accuracy: 0.2548 - val_loss: 1.5924 - val_accuracy: 0.2834
Epoch 2/6
92/92 [==============================] - 6s 66ms/step - loss: 1.5261 - accuracy: 0.3392 - val_loss: 1.5553 - val_accuracy: 0.3406
Epoch 3/6
92/92 [==============================] - 6s 65ms/step - loss: 1.5153 - accuracy: 0.3236 - val_loss: 1.5620 - val_accuracy: 0.3651
Epoch 4/6
92/92 [==============================] - 6s 65ms/step - loss: 1.4879 - accuracy: 0.3304 - val_loss: 1.5920 - val_accuracy: 0.4033
Epoch 5/6
92/92 [==============================] - 6s 65ms/step - loss: 1.4419 - accuracy: 0.3675 - val_loss: 1.5167 - val_accuracy: 0.4142
Epoch 6/6
92/92 [==============================] - 6s 68ms/step - loss: 1.4356 - accuracy: 0.3658 - val_loss: 1.5409 - val_accuracy: 0.4142


## Model 2 

In [14]:
model_2 = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [15]:
model_2.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [16]:
model_2.fit(
  train_ds,
  validation_data=val_ds,
  epochs=1
)

92/92 [==============================] - 80s 872ms/step - loss: 1.2945 - accuracy: 0.4377 - val_loss: 1.1570 - val_accuracy: 0.5259


In [17]:
!pwd

/c/ShankersDocs/EDUCATION/RICE_Bootcamp_DataAnalytics/FinalProject_Img_Recognition_Flowers/Final_RICEproject_ImageRecognition_flowers


In [18]:
import sys
print(sys.executable)a

C:\ProgramData\Anaconda3\envs\Mlearning\python.exe


In [19]:
!conda info


     active environment : Mlearning
    active env location : C:\ProgramData\Anaconda3\envs\Mlearning
            shell level : 2
       user config file : C:\Users\nairs\.condarc
 populated config files : C:\Users\nairs\.condarc
          conda version : 4.8.3
    conda-build version : 3.18.11
         python version : 3.7.6.final.0
       virtual packages : __cuda=10.2
       base environment : C:\ProgramData\Anaconda3  (writable)
           channel URLs : https://repo.anaconda.com/pkgs/main/win-64
                          https://repo.anaconda.com/pkgs/main/noarch
                          https://repo.anaconda.com/pkgs/r/win-64
                          https://repo.anaconda.com/pkgs/r/noarch
                          https://repo.anaconda.com/pkgs/msys2/win-64
                          https://repo.anaconda.com/pkgs/msys2/noarch
          package cache : C:\ProgramData\Anaconda3\pkgs
                          C:\Users\nairs\.conda\pkgs
                          C:\Users\nairs\Ap

In [23]:
model_2.save('C:/Users/nairs/Desktop/model_2')

INFO:tensorflow:Assets written to: C:/Users/nairs/Desktop/model_2\assets


## Model 3 - TRAINED MODEL

## Resource:
https://www.tensorflow.org/hub/tutorials/tf2_image_retraining

Since training a model from scratch needs a large amount of labeled data, below is a pre-trained TF2 SavedModel from TensorFlow Hub for image feature extraction, trained on the much larger and more general ImageNet dataset.

In [37]:
module_selection = ("mobilenet_v2_100_224", 224) 
handle_base, pixels = module_selection
MODULE_HANDLE ="https://tfhub.dev/google/imagenet/{}/feature_vector/4".format(handle_base)

In [38]:
MODULE_HANDLE

'https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4'

In [39]:
pixels

224

In [40]:
IMAGE_SIZE = (pixels, pixels)
print("Using {} with input size {}".format(MODULE_HANDLE, IMAGE_SIZE))

Using https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4 with input size (224, 224)


In [41]:
do_fine_tuning = False 

In [42]:
IMAGE_SIZE

(224, 224)

In [43]:
print("Building model with", MODULE_HANDLE)
model3 = tf.keras.Sequential([
    # Explicitly define the input shape so the model can be properly
    # loaded by the TFLiteConverter
    tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
    hub.KerasLayer(MODULE_HANDLE, trainable=do_fine_tuning),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(num_classes,
                          kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])

Building model with https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4


In [44]:
model3.build((None,)+IMAGE_SIZE+(3,))

In [45]:
model3.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 1280)              2257984   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 6405      
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [46]:
model3.compile(
  optimizer=tf.keras.optimizers.SGD(lr=0.005, momentum=0.9), 
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [47]:
model3.fit(
    normalized_train_ds,
    validation_data = normalized_val_ds, 
    epochs=6
)

Epoch 1/6
104/104 [==============================] - 139s 1s/step - loss: 0.6269 - accuracy: 0.7714 - val_loss: 0.3050 - val_accuracy: 0.8910
Epoch 2/6
104/104 [==============================] - 125s 1s/step - loss: 0.3416 - accuracy: 0.8834 - val_loss: 0.2900 - val_accuracy: 0.8992
Epoch 3/6
104/104 [==============================] - 124s 1s/step - loss: 0.2878 - accuracy: 0.8998 - val_loss: 0.2797 - val_accuracy: 0.9074
Epoch 4/6
104/104 [==============================] - 123s 1s/step - loss: 0.2491 - accuracy: 0.9131 - val_loss: 0.2607 - val_accuracy: 0.9128
Epoch 5/6
104/104 [==============================] - 122s 1s/step - loss: 0.2222 - accuracy: 0.9258 - val_loss: 0.2522 - val_accuracy: 0.9183
Epoch 6/6
104/104 [==============================] - 124s 1s/step - loss: 0.2113 - accuracy: 0.9304 - val_loss: 0.2471 - val_accuracy: 0.9183


In [48]:
model3.fit(
    normalized_train_ds,
    validation_data = normalized_val_ds, 
    epochs=12
)

Epoch 1/12
104/104 [==============================] - 127s 1s/step - loss: 0.2045 - accuracy: 0.9279 - val_loss: 0.2350 - val_accuracy: 0.9264
Epoch 2/12
104/104 [==============================] - 128s 1s/step - loss: 0.1850 - accuracy: 0.9364 - val_loss: 0.2288 - val_accuracy: 0.9292
Epoch 3/12
104/104 [==============================] - 133s 1s/step - loss: 0.1784 - accuracy: 0.9398 - val_loss: 0.2347 - val_accuracy: 0.9319
Epoch 4/12
104/104 [==============================] - 132s 1s/step - loss: 0.1705 - accuracy: 0.9431 - val_loss: 0.2356 - val_accuracy: 0.9264
Epoch 5/12
104/104 [==============================] - 139s 1s/step - loss: 0.1625 - accuracy: 0.9458 - val_loss: 0.2299 - val_accuracy: 0.9292
Epoch 6/12
104/104 [==============================] - 138s 1s/step - loss: 0.1594 - accuracy: 0.9455 - val_loss: 0.2290 - val_accuracy: 0.9319
Epoch 7/12
104/104 [==============================] - 139s 1s/step - loss: 0.1460 - accuracy: 0.9497 - val_loss: 0.2168 - val_accuracy: 0.9264

In [49]:
model3.fit(
    normalized_train_ds,
    validation_data = normalized_val_ds, 
    epochs=18
)

Epoch 1/18
104/104 [==============================] - 130s 1s/step - loss: 0.1316 - accuracy: 0.9552 - val_loss: 0.2259 - val_accuracy: 0.9237
Epoch 2/18
104/104 [==============================] - 132s 1s/step - loss: 0.1272 - accuracy: 0.9540 - val_loss: 0.2318 - val_accuracy: 0.9237
Epoch 3/18
104/104 [==============================] - 133s 1s/step - loss: 0.1188 - accuracy: 0.9616 - val_loss: 0.2176 - val_accuracy: 0.9292
Epoch 4/18
104/104 [==============================] - 125s 1s/step - loss: 0.1251 - accuracy: 0.9582 - val_loss: 0.2268 - val_accuracy: 0.9373
Epoch 5/18
104/104 [==============================] - 133s 1s/step - loss: 0.1147 - accuracy: 0.9640 - val_loss: 0.2351 - val_accuracy: 0.9210
Epoch 6/18
104/104 [==============================] - 130s 1s/step - loss: 0.1089 - accuracy: 0.9643 - val_loss: 0.2350 - val_accuracy: 0.9264
Epoch 7/18
104/104 [==============================] - 128s 1s/step - loss: 0.1130 - accuracy: 0.9619 - val_loss: 0.2225 - val_accuracy: 0.9264

In [50]:
model3.save('C:/Users/nairs/Desktop/mobilenet_model_90_10')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: C:/Users/nairs/Desktop/mobilenet_model_90_10\assets


INFO:tensorflow:Assets written to: C:/Users/nairs/Desktop/mobilenet_model_90_10\assets
